In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['SPD']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [4]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    if len(pmids) > 10000:
        pmids = random.choices(pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [5]:
list(zip(longforms, counts))

[('spermidine', 63),
 ('stepholidine', 29),
 ('serious psychological distress', 15),
 ('sulpiride', 15),
 ('schizotypal personality disorder', 10),
 ('sprague dawley', 9),
 ('subcorneal pustular dermatosis', 8),
 ('surfactant protein d', 7),
 ('synpolydactyly', 6),
 ('sample progression discovery', 5),
 ('styrylpyrone derivative', 5),
 ('severe plastic deformation', 3),
 ('surfactant polymer dressing', 3)]

In [7]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8890)

INFO: [2020-09-24 20:06:54] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /home/ubuntu/.indra/bio_ontology/1.2/bio_ontology.pkl


In [8]:
result = [grounding_map, names, pos_labels]

In [9]:
result

[{'sample progression discovery': 'ungrounded',
  'schizotypal personality disorder': 'MESH:D012569',
  'serious psychological distress': 'MESH:D000079225',
  'severe plastic deformation': 'ungrounded',
  'spermidine': 'CHEBI:CHEBI:16610',
  'sprague dawley': 'MESH:D017207',
  'stepholidine': 'MESH:C010513',
  'styrylpyrone derivative': 'ungrounded',
  'subcorneal pustular dermatosis': 'DOID:DOID:8508',
  'sulpiride': 'CHEBI:CHEBI:32168',
  'surfactant polymer dressing': 'ungrounded',
  'surfactant protein d': 'HGNC:10803',
  'synpolydactyly': 'DOID:DOID:0060242'},
 {'MESH:D012569': 'Schizotypal Personality Disorder',
  'MESH:D000079225': 'Psychological Distress',
  'CHEBI:CHEBI:16610': 'spermidine',
  'MESH:D017207': 'Rats, Sprague-Dawley',
  'MESH:C010513': 'stepholidine',
  'DOID:DOID:8508': 'subcorneal pustular dermatosis',
  'CHEBI:CHEBI:32168': 'sulpiride',
  'HGNC:10803': 'SFTPD',
  'DOID:DOID:0060242': 'synpolydactyly'},
 ['CHEBI:CHEBI:16610', 'MESH:C010513']]

In [10]:
grounding_map, names, pos_labels = [{'sample progression discovery': 'ungrounded',
  'schizotypal personality disorder': 'MESH:D012569',
  'serious psychological distress': 'MESH:D000079225',
  'severe plastic deformation': 'ungrounded',
  'spermidine': 'CHEBI:CHEBI:16610',
  'sprague dawley': 'MESH:D017207',
  'stepholidine': 'MESH:C010513',
  'styrylpyrone derivative': 'ungrounded',
  'subcorneal pustular dermatosis': 'DOID:DOID:8508',
  'sulpiride': 'CHEBI:CHEBI:32168',
  'surfactant polymer dressing': 'ungrounded',
  'surfactant protein d': 'HGNC:10803',
  'synpolydactyly': 'DOID:DOID:0060242'},
 {'MESH:D012569': 'Schizotypal Personality Disorder',
  'MESH:D000079225': 'Psychological Distress',
  'CHEBI:CHEBI:16610': 'spermidine',
  'MESH:D017207': 'Rats, Sprague-Dawley',
  'MESH:C010513': 'stepholidine',
  'DOID:DOID:8508': 'subcorneal pustular dermatosis',
  'CHEBI:CHEBI:32168': 'sulpiride',
  'HGNC:10803': 'SFTPD',
  'DOID:DOID:0060242': 'synpolydactyly'},
 ['CHEBI:CHEBI:16610', 'MESH:C010513']]

In [11]:
excluded_longforms = []

In [12]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [13]:
additional_entities = []

In [14]:
unambiguous_agent_texts = {}

In [26]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entities = pos_labels + additional_entities
entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in entities}

In [16]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [17]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [18]:
intersection1

[('CHEBI:CHEBI:16610', 'CHEBI:CHEBI:16610', 3850),
 ('CHEBI:CHEBI:16610', 'MESH:C010513', 0),
 ('MESH:C010513', 'CHEBI:CHEBI:16610', 0),
 ('MESH:C010513', 'MESH:C010513', 0)]

In [19]:
intersection2

[('MESH:C010513', 'CHEBI:CHEBI:16610', 0),
 ('MESH:C010513', 'MESH:C010513', 0),
 ('CHEBI:CHEBI:16610', 'CHEBI:CHEBI:16610', 22),
 ('CHEBI:CHEBI:16610', 'MESH:C010513', 0),
 ('MESH:D017207', 'CHEBI:CHEBI:16610', 0),
 ('MESH:D017207', 'MESH:C010513', 0),
 ('ungrounded', 'CHEBI:CHEBI:16610', 0),
 ('ungrounded', 'MESH:C010513', 0),
 ('DOID:DOID:8508', 'CHEBI:CHEBI:16610', 0),
 ('DOID:DOID:8508', 'MESH:C010513', 0),
 ('DOID:DOID:0060242', 'CHEBI:CHEBI:16610', 0),
 ('DOID:DOID:0060242', 'MESH:C010513', 0),
 ('MESH:D000079225', 'CHEBI:CHEBI:16610', 0),
 ('MESH:D000079225', 'MESH:C010513', 0),
 ('MESH:D012569', 'CHEBI:CHEBI:16610', 0),
 ('MESH:D012569', 'MESH:C010513', 0),
 ('HGNC:10803', 'CHEBI:CHEBI:16610', 0),
 ('HGNC:10803', 'MESH:C010513', 0),
 ('CHEBI:CHEBI:32168', 'CHEBI:CHEBI:16610', 0),
 ('CHEBI:CHEBI:32168', 'MESH:C010513', 0)]

[('HGNC:7321', ['Musculin', 'musculin'])]

In [27]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items()])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=['SPD', 'spermidine', 'Spermidine'])
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) > 5])

In [28]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-09-24 20:41:37] /adeft/PythonRepos/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-09-24 20:41:56] /adeft/PythonRepos/adeft/adeft/modeling/classify.py - Best f1 score of 0.9910350462025803 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [29]:
classifier.stats

{'label_distribution': {'MESH:C010513': 25,
  'CHEBI:CHEBI:16610': 2056,
  'MESH:D017207': 5,
  'ungrounded': 7,
  'DOID:DOID:8508': 6,
  'DOID:DOID:0060242': 5,
  'MESH:D000079225': 8,
  'MESH:D012569': 8,
  'HGNC:10803': 5,
  'CHEBI:CHEBI:32168': 1},
 'f1': {'mean': 0.991035, 'std': 0.004081},
 'precision': {'mean': 0.986724, 'std': 0.005681},
 'recall': {'mean': 0.996155, 'std': 0.001924},
 'CHEBI:CHEBI:32168': {'f1': {'mean': 0.0, 'std': 0.0},
  'pr': {'mean': 0.0, 'std': 0.0},
  'rc': {'mean': 0.0, 'std': 0.0}},
 'HGNC:10803': {'f1': {'mean': 0.4, 'std': 0.489898},
  'pr': {'mean': 0.4, 'std': 0.489898},
  'rc': {'mean': 0.4, 'std': 0.489898}},
 'ungrounded': {'f1': {'mean': 0.4, 'std': 0.489898},
  'pr': {'mean': 0.4, 'std': 0.489898},
  'rc': {'mean': 0.4, 'std': 0.489898}},
 'MESH:D000079225': {'f1': {'mean': 0.533333, 'std': 0.452155},
  'pr': {'mean': 0.5, 'std': 0.447214},
  'rc': {'mean': 0.6, 'std': 0.489898}},
 'DOID:DOID:0060242': {'f1': {'mean': 0.8, 'std': 0.4},
  'pr'

In [30]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [31]:
disamb.dump(model_name, results_path)

In [32]:
print(disamb.info())

Disambiguation model for SPD

Produces the disambiguations:
	Psychological Distress	MESH:D000079225
	Rats, Sprague-Dawley	MESH:D017207
	SFTPD	HGNC:10803
	Schizotypal Personality Disorder	MESH:D012569
	spermidine*	CHEBI:CHEBI:16610
	stepholidine*	MESH:C010513
	subcorneal pustular dermatosis	DOID:DOID:8508
	sulpiride	CHEBI:CHEBI:32168
	synpolydactyly	DOID:DOID:0060242

Class level metrics:
--------------------
Grounding                       	Count	F1     
                      spermidine*	2056	  0.993
                    stepholidine*	  25	0.82984
          Psychological Distress	   8	0.53333
Schizotypal Personality Disorder	   8	0.86667
                      Ungrounded	   7	    0.4
  subcorneal pustular dermatosis	   6	0.73333
            Rats, Sprague-Dawley	   5	    0.2
                  synpolydactyly	   5	    0.8
                           SFTPD	   5	    0.4
                       sulpiride	   1	    0.0

Weighted Metrics:
-----------------
	F1 score:	0.99104
	Precision:	0.98672
	Re

In [33]:
model_to_s3(disamb)